In [11]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os 
from warnings import simplefilter
import pandas as pd
from imblearn.over_sampling import SMOTE  

In [12]:
import models
import class_sampling
import train
import metric_utils
import inference
import loss_fns
import torchvision.ops 

In [13]:
NUM_CLASSES = 10
n_epochs = 30
batch_size_train = 64
batch_size_test = 1000
momentum = 0

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

NUM_CLASSES_REDUCED = 2
nums = (0, 1)
ratio = (100, 1)

CLASS_LABELS = {'airplane': 0,
                 'automobile': 1,
                 'bird': 2,
                 'cat': 3,
                 'deer': 4,
                 'dog': 5,
                 'frog': 6,
                 'horse': 7,
                 'ship': 8,
                 'truck': 9}


simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=DeprecationWarning)

In [14]:
col_names = ["name", 
            "num_classes", 
            "classes_used", 
            "ratio", 
            "learning_rate", 
            "mean_0", "variance_0",
            "mean_10", "variance_10",
            "mean_20", "variance_20",
            "mean_30", "variance_30",
          #   "mean_40", "variance_40",
          #   "mean_50", "variance_50",
             "cap", "normalization"]

rows = []

In [15]:
norm=False

if norm:
    transform=torchvision.transforms.Compose([torchvision.transforms.Normalize(mean=[143.8888, 127.1705, 117.5357], std=[69.8313, 64.5137, 66.9933])])
else:
   # transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
    transform=None

train_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=True, download=True,
                             transform=transform)  


test_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=False, download=True,
                             transform=transform)  

train_CIFAR10.data = train_CIFAR10.data.reshape(50000, 3, 32, 32)
test_CIFAR10.data = test_CIFAR10.data.reshape(10000, 3, 32, 32)

    
reduced_train_CIFAR10 = class_sampling.Reduce(train_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True, transform=transform)
reduced_test_CIFAR10 = class_sampling.Reduce(test_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True, transform=transform)

ratio_train_CIFAR10 = class_sampling.Ratio(train_CIFAR10, NUM_CLASSES_REDUCED, ratio, nums=nums, transform=transform)

triplet_train_CIFAR10 = class_sampling.ForTripletLoss(reduced_train_CIFAR10, smote=False, transform=transform, num_classes=2)
triplet_ratio_train_CIFAR10 = class_sampling.ForTripletLoss(ratio_train_CIFAR10, smote=False, transform=transform, num_classes=2)

smote_train_CIFAR10 = class_sampling.Smote(ratio_train_CIFAR10, 5000 * NUM_CLASSES_REDUCED, transform=transform)
triplet_smote_train_CIFAR10 = class_sampling.ForTripletLoss(smote_train_CIFAR10, smote=True, transform=transform, num_classes=2)

Files already downloaded and verified
Files already downloaded and verified


In [16]:
targets = ratio_train_CIFAR10.labels 

class_count = np.unique(targets, return_counts=True)[1]
print(class_count)

weight = 1. / class_count

samples_weight = weight[targets]
samples_weight = torch.from_numpy(samples_weight)
oversampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(max(class_count) * NUM_CLASSES_REDUCED), replacement=True)
sampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)
undersampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(min(class_count) * NUM_CLASSES_REDUCED), replacement=False)
undersampler_smote = torch.utils.data.WeightedRandomSampler(samples_weight, int(min(class_count) * 50 * NUM_CLASSES_REDUCED), replacement=False)
weight *= class_count[0]

[5000   50]


In [17]:
train_loader = DataLoader(train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_reduced = DataLoader(reduced_train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_ratio = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True) 

train_loader_oversampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=oversampler)

train_loader_undersampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=undersampler)

train_loader_sampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=sampler)

train_loader_smote = DataLoader(smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_smote_undersampled = DataLoader(smote_train_CIFAR10, batch_size=batch_size_train, sampler=undersampler_smote)

train_loader_tripletloss = DataLoader(triplet_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss_ratio = DataLoader(triplet_ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss_smote = DataLoader(triplet_smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

test_loader_reduced = DataLoader(reduced_test_CIFAR10, batch_size=batch_size_test, shuffle=True)

In [24]:
# 2 CLASS normal

momentum=0
learning_rates = [5e-2, 1e-2, 1e-3, 5e-4]

learning_rate_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_reduced, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["normal", 2, nums, (1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)



Test set: Avg. loss: 0.0006989752054214478, AUC: 0.306551



KeyboardInterrupt: 

In [47]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []


In [48]:
# 2 CLASS ratio

momentum=0
learning_rates = [5e-2, 1e-2, 1e-3, 5e-4, 5e-3]

learning_rate_aucs = []
learning_rate_train_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_ratio, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                _, train_auc = metric_utils.auc_sigmoid(train_loader_ratio, network) 
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["ratio", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)



Test set: Avg. loss: 0.0006964986324310303, AUC: 0.36889099999999997


Test set: Avg. loss: 0.0020484524965286256, AUC: 0.7839530000000001


Test set: Avg. loss: 0.0008004228307837897, AUC: 0.795284


Test set: Avg. loss: 0.0019669443964958192, AUC: 0.8054380000000001


Test set: Avg. loss: 0.0007447379432839922, AUC: 0.845868


Test set: Avg. loss: 0.001964762210845947, AUC: 0.8220825


Test set: Avg. loss: 0.0006831325510387668, AUC: 0.8975879999999999


Test set: Avg. loss: 0.0007002675533294678, AUC: 0.29741149999999994


Test set: Avg. loss: 0.002268509268760681, AUC: 0.7684610000000001


Test set: Avg. loss: 0.000813441066587768, AUC: 0.779452


Test set: Avg. loss: 0.001961479306221008, AUC: 0.8012969999999999


Test set: Avg. loss: 0.0007347470497952239, AUC: 0.8548


Test set: Avg. loss: 0.001975426435470581, AUC: 0.8130399999999999


Test set: Avg. loss: 0.0006815541962548943, AUC: 0.888204


Test set: Avg. loss: 0.0006969135999679566, AUC: 0.396233


Test set: Avg. loss: 0.


Test set: Avg. loss: 0.0022208908796310426, AUC: 0.738314


Test set: Avg. loss: 0.0008432061213188537, AUC: 0.747452


Test set: Avg. loss: 0.0021523762941360472, AUC: 0.7596109999999999


Test set: Avg. loss: 0.0008099005716198152, AUC: 0.772908


Test set: Avg. loss: 0.0006932830810546875, AUC: 0.540304


Test set: Avg. loss: 0.002492153882980347, AUC: 0.69481


Test set: Avg. loss: 0.0008906742870475691, AUC: 0.70906


Test set: Avg. loss: 0.002333818554878235, AUC: 0.721683


Test set: Avg. loss: 0.0008443353652474608, AUC: 0.739868


Test set: Avg. loss: 0.002207614302635193, AUC: 0.743633


Test set: Avg. loss: 0.000816104515907493, AUC: 0.7657440000000001


Test set: Avg. loss: 0.000693915456533432, AUC: 0.470027


Test set: Avg. loss: 0.0019597501158714293, AUC: 0.6219365000000001


Test set: Avg. loss: 0.0010087698723862666, AUC: 0.6026600000000001


Test set: Avg. loss: 0.0025770732164382935, AUC: 0.649457


Test set: Avg. loss: 0.0009415403474583336, AUC: 0.63322


Test se


Test set: Avg. loss: 0.0010332340002059937, AUC: 0.639335


Test set: Avg. loss: 0.002572380872351108, AUC: 0.644908


Test set: Avg. loss: 0.0018369460701942444, AUC: 0.637613


Test set: Avg. loss: 0.0010801145913872387, AUC: 0.641616


Test set: Avg. loss: 0.0022674994468688966, AUC: 0.640206


Test set: Avg. loss: 0.0009589608891470597, AUC: 0.643248


Test set: Avg. loss: 0.0006972042322158814, AUC: 0.3433105


Test set: Avg. loss: 0.0014003113508224488, AUC: 0.540345


Test set: Avg. loss: 0.001675022819844803, AUC: 0.536408


Test set: Avg. loss: 0.002195003390312195, AUC: 0.5878695


Test set: Avg. loss: 0.0009807692990075833, AUC: 0.581788


Test set: Avg. loss: 0.0025299243927001955, AUC: 0.612595


Test set: Avg. loss: 0.0009592226822630013, AUC: 0.60672


Test set: Avg. loss: 0.0006968080401420593, AUC: 0.42184750000000004


Test set: Avg. loss: 0.0014318109154701233, AUC: 0.500686


Test set: Avg. loss: 0.0016474896108750069, AUC: 0.46987599999999996


Test set: Avg. loss

In [49]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [14]:
# 2 CLASS oversampled

momentum=0
learning_rates = [5e-2, 1e-2, 1e-3, 5e-3, 1e-4]

learning_rate_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_oversampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["oversampled", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)


Test set: Avg. loss: 0.0006984597146511078, AUC: 0.3728135



KeyboardInterrupt: 

In [51]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [52]:
# 2 CLASS undersampled

momentum=0
learning_rates = [5e-2, 1e-2, 1e-3, 5e-3, 1e-4]

learning_rate_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_undersampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["undersampled", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)



Test set: Avg. loss: 0.0006928627490997315, AUC: 0.5610565


Test set: Avg. loss: 0.0006705484390258789, AUC: 0.731087


Test set: Avg. loss: 0.0006308182775974274, AUC: 0.7689


Test set: Avg. loss: 0.0006022096574306488, AUC: 0.792463


Test set: Avg. loss: 0.0006950775682926178, AUC: 0.4702115


Test set: Avg. loss: 0.0006825314462184907, AUC: 0.7292394999999999


Test set: Avg. loss: 0.0006524708569049835, AUC: 0.754748


Test set: Avg. loss: 0.0006467648446559906, AUC: 0.7728849999999999


Test set: Avg. loss: 0.0006955726444721221, AUC: 0.3766905


Test set: Avg. loss: 0.0007012077569961548, AUC: 0.601467


Test set: Avg. loss: 0.0006722729504108429, AUC: 0.7478565


Test set: Avg. loss: 0.0006584925651550293, AUC: 0.764774


Test set: Avg. loss: 0.0006918634176254272, AUC: 0.548633


Test set: Avg. loss: 0.0006640854477882385, AUC: 0.7368910000000001


Test set: Avg. loss: 0.0006435896456241608, AUC: 0.772193


Test set: Avg. loss: 0.0006132080554962159, AUC: 0.805193


Test se


Test set: Avg. loss: 0.000681488573551178, AUC: 0.661619


Test set: Avg. loss: 0.0006757974326610565, AUC: 0.6790075000000001


Test set: Avg. loss: 0.0006896138191223144, AUC: 0.610887


Test set: Avg. loss: 0.0006893069446086884, AUC: 0.613855


Test set: Avg. loss: 0.0006885486543178558, AUC: 0.633282


Test set: Avg. loss: 0.0006888088881969452, AUC: 0.6441379999999999


Test set: Avg. loss: 0.000698607712984085, AUC: 0.47644650000000005


Test set: Avg. loss: 0.0006970592141151428, AUC: 0.5706005000000001


Test set: Avg. loss: 0.0006956733465194702, AUC: 0.6301249999999999


Test set: Avg. loss: 0.0006951669156551361, AUC: 0.6572


Test set: Avg. loss: 0.0006925795376300812, AUC: 0.575475


Test set: Avg. loss: 0.0006951481699943542, AUC: 0.5105139999999999


Test set: Avg. loss: 0.0006970000267028808, AUC: 0.4651495


Test set: Avg. loss: 0.000699210911989212, AUC: 0.424679


Test set: Avg. loss: 0.0006938398480415344, AUC: 0.49060099999999995


Test set: Avg. loss: 0.00069005

In [53]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [54]:
# 2 Class Weighted Loss 

momentum=0
learning_rates = [5e-2, 1e-2, 5e-3, 1e-3, 5e-4]

learning_rate_aucs = []

loss_fn_args = {}
loss_fn_args['pos_weight'] = torch.tensor([weight[1]])


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_ratio, network, optimizer, verbose=False, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                _, train_auc = metric_utils.auc_sigmoid(train_loader_ratio, network) 
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["weighted", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)



Test set: Avg. loss: 0.0006929865777492523, AUC: 0.6035210000000001


Test set: Avg. loss: 0.0006941011548042297, AUC: 0.5


Test set: Avg. loss: 0.01152815311261923, AUC: 0.5


Test set: Avg. loss: 0.0006933517456054688, AUC: 0.5


Test set: Avg. loss: 0.010536302033037242, AUC: 0.5


Test set: Avg. loss: 0.0006934399306774139, AUC: 0.5


Test set: Avg. loss: 0.01047680665950964, AUC: 0.5


Test set: Avg. loss: 0.0006829564273357391, AUC: 0.7606660000000001


Test set: Avg. loss: 0.0007110981047153472, AUC: 0.513034


Test set: Avg. loss: 0.008160775073684089, AUC: 0.4939


Test set: Avg. loss: 0.0006991162598133087, AUC: 0.5


Test set: Avg. loss: 0.009259643141586001, AUC: 0.5


Test set: Avg. loss: 0.0006932158470153809, AUC: 0.5


Test set: Avg. loss: 0.011024211881184342, AUC: 0.5


Test set: Avg. loss: 0.0006900024712085724, AUC: 0.6275345


Test set: Avg. loss: 0.0006032556593418122, AUC: 0.6915875


Test set: Avg. loss: 0.009716101306499822, AUC: 0.67254


Test set: Avg. loss


Test set: Avg. loss: 0.0005637988150119782, AUC: 0.828475


Test set: Avg. loss: 0.00714115304521995, AUC: 0.9039240000000001


Test set: Avg. loss: 0.0005115714669227601, AUC: 0.872054


Test set: Avg. loss: 0.0058704357690150194, AUC: 0.9506


Test set: Avg. loss: 0.0005990622639656067, AUC: 0.854916


Test set: Avg. loss: 0.004999826568188054, AUC: 0.9478759999999999


Test set: Avg. loss: 0.0006893006265163422, AUC: 0.592972


Test set: Avg. loss: 0.0005413131713867187, AUC: 0.838742


Test set: Avg. loss: 0.009401522638774154, AUC: 0.873108


Test set: Avg. loss: 0.0005082505941390992, AUC: 0.8643450000000001


Test set: Avg. loss: 0.007597315765843533, AUC: 0.93034


Test set: Avg. loss: 0.0005349463224411011, AUC: 0.819852


Test set: Avg. loss: 0.005960197472336269, AUC: 0.952184


Test set: Avg. loss: 0.0006917398273944855, AUC: 0.5357335000000001


Test set: Avg. loss: 0.0005320677161216736, AUC: 0.872421


Test set: Avg. loss: 0.007882021895729669, AUC: 0.885336


Test set:


Test set: Avg. loss: 0.0005055390298366547, AUC: 0.849865


Test set: Avg. loss: 0.008350928868397627, AUC: 0.8736079999999999


Test set: Avg. loss: 0.0006963136792182922, AUC: 0.39968349999999997


Test set: Avg. loss: 0.0005984225273132324, AUC: 0.771389


Test set: Avg. loss: 0.009001542176350508, AUC: 0.78562


Test set: Avg. loss: 0.000558992862701416, AUC: 0.8067799999999999


Test set: Avg. loss: 0.008595412568290635, AUC: 0.829488


Test set: Avg. loss: 0.000538530021905899, AUC: 0.82298


Test set: Avg. loss: 0.007287021987509019, AUC: 0.8586119999999999


Test set: Avg. loss: 0.0006968409717082977, AUC: 0.38355749999999994


Test set: Avg. loss: 0.0006436823904514313, AUC: 0.803137


Test set: Avg. loss: 0.01135210377154964, AUC: 0.7909520000000001


Test set: Avg. loss: 0.000583594560623169, AUC: 0.8077380000000001


Test set: Avg. loss: 0.01014432730060993, AUC: 0.820932


Test set: Avg. loss: 0.0005280269980430603, AUC: 0.833491


Test set: Avg. loss: 0.00862787301587586

In [55]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [8]:
# 2 CLASS SMOTE

momentum=0
learning_rates = [5e-2, 1e-2, 5e-3, 1e-3, 1e-4]

learning_rate_aucs = []

loss_fn_args = {}
loss_fn_args['loss_cap'] = None


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid_with_smote(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["smote", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)



Test set: Avg. loss: 0.001968830943107605, AUC: 0.6361665000000001


Test set: Avg. loss: 0.0006931888163089752, AUC: 0.5


Test set: Avg. loss: 0.0006931563913822174, AUC: 0.5


Test set: Avg. loss: 0.0006931565403938293, AUC: 0.5


Test set: Avg. loss: 0.0032240989208221437, AUC: 0.38664699999999996


Test set: Avg. loss: 0.000693162739276886, AUC: 0.5


Test set: Avg. loss: 0.0006931797564029694, AUC: 0.5


Test set: Avg. loss: 0.0006931637823581696, AUC: 0.5


Test set: Avg. loss: 0.006873347997665405, AUC: 0.683021


Test set: Avg. loss: 0.000693148136138916, AUC: 0.5


Test set: Avg. loss: 0.0006931990087032318, AUC: 0.5


Test set: Avg. loss: 0.0006931540668010712, AUC: 0.5


Test set: Avg. loss: 0.001451125681400299, AUC: 0.47390000000000004


Test set: Avg. loss: 0.0006931474804878234, AUC: 0.5


Test set: Avg. loss: 0.0006931490898132324, AUC: 0.5


Test set: Avg. loss: 0.0006931617856025696, AUC: 0.5


Test set: Avg. loss: 0.0031172895431518556, AUC: 0.6896909999999999


Te


Test set: Avg. loss: 0.00241806423664093, AUC: 0.3173455


Test set: Avg. loss: 0.0006629388034343719, AUC: 0.7816734999999999


Test set: Avg. loss: 0.0005770286917686462, AUC: 0.8724530000000001


Test set: Avg. loss: 0.0005904681980609894, AUC: 0.892585


Test set: Avg. loss: 0.001278232514858246, AUC: 0.507354


Test set: Avg. loss: 0.0006930362582206726, AUC: 0.5019979999999999


Test set: Avg. loss: 0.0006615914404392243, AUC: 0.7755829999999999


Test set: Avg. loss: 0.0007022977471351624, AUC: 0.6703819999999999


Test set: Avg. loss: 0.0020855687856674195, AUC: 0.4122905


Test set: Avg. loss: 0.0006579199731349945, AUC: 0.7659765


Test set: Avg. loss: 0.000646492600440979, AUC: 0.8105619999999999


Test set: Avg. loss: 0.0006940586864948272, AUC: 0.7869459999999999


Test set: Avg. loss: 0.0013735867142677307, AUC: 0.58304


Test set: Avg. loss: 0.000610756278038025, AUC: 0.8491740000000001


Test set: Avg. loss: 0.0006886270344257355, AUC: 0.737814


Test set: Avg. loss: 0

In [9]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [56]:
# 2 Class Capped SMOTE 

momentum=0
learning_rates = [1e-3,1e-2, 5e-3]


cap_aucs = []

caps = [1, 5, 10]

for cap in caps:
    
    loss_fn_args = {}
    loss_fn_args['loss_cap'] = cap
    
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNet(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
            model_aucs.append(auc)
            for epoch in range(n_epochs):
                _, _ = train.train_sigmoid_with_smote(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = caps[c]
    for i in range(len(learning_rates)): 
        row = ["capped_smote", 2, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

print(rows)



Test set: Avg. loss: 0.0006927153766155242, AUC: 0.520336


Test set: Avg. loss: 0.0006058035492897034, AUC: 0.7684874999999999



KeyboardInterrupt: 

In [ ]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = (col_names))

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [10]:
# 2 CLASS Focal Loss
# this code might not work 

momentum=0
learning_rates = [1e-4, 1e-5]

learning_rate_aucs = []

loss_fn_args = {}


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_ratio, network, optimizer, verbose=False, loss_fn=loss_fns.SigmoidFocalLoss, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["focal_loss", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)



Test set: Avg. loss: 0.0012810109853744506, AUC: 0.4544585



TypeError: unsupported operand type(s) for *: 'Tensor' and 'NoneType'

In [38]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

NameError: name 'df2' is not defined

In [ ]:
# distance + capped loss
momentum=0
learning_rates = [1e-3, 5e-3, 5e-4, 1e-4]


    
loss_fn_args = {}
loss_cap = 5.0
loss_fn_args['distance'] = 'euclidean'

start_epoch = 2

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNetWithEmbeddings(2)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True) 
        model_aucs.append(auc)
        for epoch in range(start_epoch):
            loss_fn_args['loss_cap'] = None
            _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
        for epoch in range(start_epoch, n_epochs + 1):
            loss_fn_args['loss_cap'] = loss_cap
            _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)


for i in range(len(learning_rates)): 
    row = ["distance_capped_smote", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], loss_fn_args['loss_cap'], norm]
    rows.append(row)


Test set: Avg. loss: 0.005698421239852905, AUC: 0.3140615


Test set: Avg. loss: 0.0006708012223243713, AUC: 0.7778695


Test set: Avg. loss: 0.0006707127690315246, AUC: 0.787516


Test set: Avg. loss: 0.0005773176550865173, AUC: 0.849029


Test set: Avg. loss: 0.004301182508468628, AUC: 0.722232


Test set: Avg. loss: 0.0006169197559356689, AUC: 0.8504210000000001


Test set: Avg. loss: 0.000606573611497879, AUC: 0.857934


Test set: Avg. loss: 0.0006615423262119293, AUC: 0.846602


Test set: Avg. loss: 0.0013060729503631592, AUC: 0.33644399999999997


Test set: Avg. loss: 0.0006230704486370087, AUC: 0.8254634999999999


Test set: Avg. loss: 0.0006484315097332001, AUC: 0.8357224999999999


Test set: Avg. loss: 0.0006770155727863312, AUC: 0.8425440000000001


Test set: Avg. loss: 0.001565246045589447, AUC: 0.39420900000000003


Test set: Avg. loss: 0.0006965601146221161, AUC: 0.859681


Test set: Avg. loss: 0.0006382490396499633, AUC: 0.871605


Test set: Avg. loss: 0.0010520057082176

In [ ]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [ ]:
# cosine distance + capped loss
momentum=0
learning_rates = [1e-3, 5e-3, 5e-4, 1e-4, 5e-5]


    
loss_fn_args = {}
loss_fn_args['loss_cap'] = 5.0
loss_fn_args['distance'] = 'cosine'

start_epoch = 2

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNetWithEmbeddings(2)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True) 
        model_aucs.append(auc)
        for epoch in range(start_epoch):
            loss_fn_args['loss_cap'] = None
            _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
        for epoch in range(start_epoch, n_epochs + 1):
            _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)


for i in range(len(learning_rates)): 
    row = ["cosine_distance_capped_smote", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], loss_fn_args['loss_cap'], norm]
    rows.append(row)

In [ ]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [8]:
# 2 class triplet loss no ratio 
# no smote 


momentum=0
learning_rates = [(5e-6, 1e-3), (1e-6, 5e-4), (1e-6, 1e-4), (5e-6, 5e-4)]

learning_rate_aucs = []

    

for learning_rate in learning_rates:
    aucs = []
    for i in range(10): 
        model_aucs = []
        embed_network = models.ConvNetOnlyEmbeddings(2)
        linear_probe = models.ConvNetLinearProbe(2)
        complete_network = models.CompleteConvNet(embed_network, linear_probe)
        embed_optimizer = optim.SGD(embed_network.parameters(), lr=learning_rate[0], momentum=momentum)
        linear_optimizer = optim.SGD(complete_network.parameters(), lr=learning_rate[1], momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True) 
        model_aucs.append(auc)
        for epoch in range(15):
            _, train_losses = train.train_triplet_loss(epoch, train_loader_tripletloss, embed_network, embed_optimizer, verbose=False)
            print("Train loss: " + str(np.mean(np.array(train_losses))))
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_reduced, complete_network, linear_optimizer, verbose=False)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True)
            if (epoch + 1) % 10 == 0: 
                model_aucs.append(auc) 
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)
    
learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)
    
    



for i in range(len(learning_rates)): 
    row = ["triplet_loss", 2, nums, (1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], 0.0, norm]
    rows.append(row)


Test set: Avg. loss: 0.004998538017272949, AUC: 0.5778245



RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor []], which is output 0 of ReluBackward0, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [9]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [11]:
# 2 class triplet loss with capped SMOTE 

# note: sometimes can get a very high accuracy but may diverge

momentum=0
learning_rates = [(5e-6, 1e-3), (1e-6, 5e-4), (1e-6, 1e-4), (5e-6, 5e-4)]

learning_rate_aucs = []

loss_fn_args = {}
loss_fn_args['loss_cap'] = 5.0
loss_fn_args['distance'] = 'euclidean'


for learning_rate in learning_rates:
    aucs = []
    for i in range(10): 
        model_aucs = []
        embed_network = models.ConvNetOnlyEmbeddings(2)
        linear_probe = models.ConvNetLinearProbe(2)
        complete_network = models.CompleteConvNet(embed_network, linear_probe)
        embed_optimizer = optim.SGD(embed_network.parameters(), lr=learning_rate[0], momentum=momentum)
        linear_optimizer = optim.SGD(complete_network.parameters(), lr=learning_rate[1], momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True) 
        model_aucs.append(auc)
        for epoch in range(15):
            _, train_losses = train.train_triplet_loss_smote(epoch, train_loader_tripletloss_smote, embed_network, embed_optimizer, verbose=False)
            print("Train loss: " + str(np.mean(np.array(train_losses))))
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_reduced, complete_network, linear_optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True)
            if (epoch + 1) % 10 == 0: 
                model_aucs.append(auc) 
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)


learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)



for i in range(len(learning_rates)): 
    row = ["triplet_loss", 2, nums, (1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], 0.0, norm]
    rows.append(row)


Test set: Avg. loss: 0.000679101675748825, AUC: 0.7387214999999999



RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor []], which is output 0 of ReluBackward0, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [11]:
# 2 class triplet loss with ratio 
# no smote 

momentum=0
learning_rates = [(1e-7, 1e-5)]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(3): 
        model_aucs = []
        embed_network = models.ConvNetOnlyEmbeddings(2)
        linear_probe = models.ConvNetLinearProbe(2)
        complete_network = models.CompleteConvNet(embed_network, linear_probe)
        embed_optimizer = optim.SGD(embed_network.parameters(), lr=learning_rate[0], momentum=momentum)
        linear_optimizer = optim.SGD(complete_network.parameters(), lr=learning_rate[1], momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True) 
        model_aucs.append(auc)
        for epoch in range(15):
            _, train_losses = train.train_triplet_loss(epoch, train_loader_tripletloss_ratio, embed_network, embed_optimizer, verbose=False)
            print("Train loss: " + str(np.mean(np.array(train_losses))))
        for epoch in range(50):
            _, _ = train.train_linear_probe(epoch, train_loader_reduced, complete_network, linear_optimizer, verbose=False)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True)
            if (epoch + 1) % 10 == 0: 
                model_aucs.append(auc) 
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)


learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["triplet_loss", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], 
           auc_mean[i][4], auc_variance[i][4],
           auc_mean[i][5], auc_variance[i][5],
           None]
    rows.append(row)


Test set: Avg. loss: 0.0007769797742366791, AUC: 0.581019

Train loss: 16.096790712090986
Train loss: 7.95101783849016
Train loss: 1.6742434924161886
Train loss: 0.0
Train loss: 6.369154097158698
Train loss: 1.0685905746266813
Train loss: 0.9919730077815961
Train loss: 4.3510085480122624
Train loss: 0.0
Train loss: 0.0
Train loss: 0.0
Train loss: 1.729099128819719
Train loss: 0.0
Train loss: 2.9074361294130737
Train loss: 0.0

Test set: Avg. loss: 0.0007081505060195923, AUC: 0.595766


Test set: Avg. loss: 0.0006918745040893554, AUC: 0.599685


Test set: Avg. loss: 0.0006822091042995453, AUC: 0.606827


Test set: Avg. loss: 0.0006758022308349609, AUC: 0.614831


Test set: Avg. loss: 0.0006717154681682586, AUC: 0.620797


Test set: Avg. loss: 0.0006670982539653778, AUC: 0.6299465


Test set: Avg. loss: 0.0006633270978927612, AUC: 0.641555


Test set: Avg. loss: 0.0006615970730781555, AUC: 0.647514


Test set: Avg. loss: 0.0006594435572624207, AUC: 0.6563729999999999


Test set: Avg. lo


Test set: Avg. loss: 0.0006901566684246063, AUC: 0.5230110000000001


Test set: Avg. loss: 0.0006899235546588897, AUC: 0.5261325


Test set: Avg. loss: 0.0006896623373031616, AUC: 0.5253125


Test set: Avg. loss: 0.0006894396543502808, AUC: 0.5309490000000001


Test set: Avg. loss: 0.0006897644698619842, AUC: 0.52842


Test set: Avg. loss: 0.0006899001598358154, AUC: 0.525083


Test set: Avg. loss: 0.0006898137331008911, AUC: 0.5242764999999999


Test set: Avg. loss: 0.0006899007260799408, AUC: 0.5234755


Test set: Avg. loss: 0.0006897071897983551, AUC: 0.5284414999999999


Test set: Avg. loss: 0.0006894522905349732, AUC: 0.5316019999999999


Test set: Avg. loss: 0.0006894703507423401, AUC: 0.5321385000000001


Test set: Avg. loss: 0.0006892255246639252, AUC: 0.5359525


Test set: Avg. loss: 0.0006892721951007843, AUC: 0.5328990000000001


Test set: Avg. loss: 0.0006891983449459076, AUC: 0.537638


Test set: Avg. loss: 0.0006890941560268402, AUC: 0.5369265000000001


Test set: Avg. l

In [12]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

ValueError: 14 columns passed, passed data had 18 columns

In [31]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [47]:
# capped smote using triplet loss

momentum=0
learning_rates = [5e-3, 1e-3, 1e-2]

loss_fn_args = {}


cap_aucs = []

start_epoch = 2

loss_caps = [1, 5, 10]

for loss_cap in loss_caps:
    
    learning_rate_aucs = []
    
    for learning_rate in learning_rates:
        aucs = []
        for i in range(10): 
            print(loss_cap)
            model_aucs = []
            network = models.ConvNetWithEmbeddings(2)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(start_epoch):
                loss_fn_args['loss_cap'] = None
                _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
            for epoch in range(start_epoch, n_epochs + 1):
                loss_fn_args['loss_cap'] = loss_cap
                _, _ = train.train_triplet_capped_loss(epoch, train_loader_tripletloss_smote, network, optimizer, verbose=False, cap_calc=loss_fns.TripletLoss,loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)
    
    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
        
    cap_aucs.append([auc_mean, auc_variance])



    


for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = loss_caps[c]
    for i in range(len(learning_rates)): 
        row = ["triplet_loss_capped_smote", 2, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

1

Test set: Avg. loss: 0.0006910498738288879, AUC: 0.616086


Test set: Avg. loss: 0.0007207183837890625, AUC: 0.832752


Test set: Avg. loss: 0.000947897344827652, AUC: 0.865763


Test set: Avg. loss: 0.000987816333770752, AUC: 0.872393

1

Test set: Avg. loss: 0.0006940085291862488, AUC: 0.5015175000000001


Test set: Avg. loss: 0.0007139445841312409, AUC: 0.83234


Test set: Avg. loss: 0.000874194860458374, AUC: 0.866993


Test set: Avg. loss: 0.001173335611820221, AUC: 0.856846

1

Test set: Avg. loss: 0.0006919935345649719, AUC: 0.6351335


Test set: Avg. loss: 0.0007935252189636231, AUC: 0.8241814999999999


Test set: Avg. loss: 0.0012041029930114746, AUC: 0.8379819999999999


Test set: Avg. loss: 0.0011449605822563172, AUC: 0.8504

1

Test set: Avg. loss: 0.0006936784386634827, AUC: 0.5426825


Test set: Avg. loss: 0.0007189079523086548, AUC: 0.841718


Test set: Avg. loss: 0.0009975289702415465, AUC: 0.854266


Test set: Avg. loss: 0.0008743861317634582, AUC: 0.876096

1

Test


Test set: Avg. loss: 0.0007955302298069, AUC: 0.821545


Test set: Avg. loss: 0.0008325413167476654, AUC: 0.8549479999999999


Test set: Avg. loss: 0.000945974737405777, AUC: 0.860855

5

Test set: Avg. loss: 0.0006943807899951935, AUC: 0.45884


Test set: Avg. loss: 0.0007042157351970673, AUC: 0.823237


Test set: Avg. loss: 0.0010037005841732026, AUC: 0.8432679999999999


Test set: Avg. loss: 0.001122573137283325, AUC: 0.8510929999999999

5

Test set: Avg. loss: 0.0006971780955791474, AUC: 0.4324415


Test set: Avg. loss: 0.0008017181158065796, AUC: 0.8170700000000001


Test set: Avg. loss: 0.0011431435942649841, AUC: 0.8431079999999999


Test set: Avg. loss: 0.0011760561466217041, AUC: 0.8444419999999999

5

Test set: Avg. loss: 0.000689770132303238, AUC: 0.664884


Test set: Avg. loss: 0.0008555488288402558, AUC: 0.8221830000000001


Test set: Avg. loss: 0.0011109410524368286, AUC: 0.830791


Test set: Avg. loss: 0.0011780904531478883, AUC: 0.85151

5

Test set: Avg. loss: 0.00069


Test set: Avg. loss: 0.0010575720071792603, AUC: 0.847473

10

Test set: Avg. loss: 0.0007015173435211182, AUC: 0.2960235


Test set: Avg. loss: 0.0008149043023586274, AUC: 0.8215784999999999


Test set: Avg. loss: 0.0009621008634567261, AUC: 0.8420889999999999


Test set: Avg. loss: 0.001291257917881012, AUC: 0.838169

10

Test set: Avg. loss: 0.0006914902627468109, AUC: 0.5626935


Test set: Avg. loss: 0.0006894059777259827, AUC: 0.82719


Test set: Avg. loss: 0.0007271471023559571, AUC: 0.86903


Test set: Avg. loss: 0.000951990693807602, AUC: 0.8576270000000001

10

Test set: Avg. loss: 0.000693727970123291, AUC: 0.519144


Test set: Avg. loss: 0.0006186171770095826, AUC: 0.8489500000000001


Test set: Avg. loss: 0.0010045465230941772, AUC: 0.8496060000000001


Test set: Avg. loss: 0.0010542113780975342, AUC: 0.8580730000000001

10

Test set: Avg. loss: 0.0006983237862586975, AUC: 0.305317


Test set: Avg. loss: 0.0007065722346305847, AUC: 0.8331059999999999


Test set: Avg. loss:

IndexError: list index out of range

In [49]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [52]:
# capped smote using triplet loss w/ average 

momentum=0
learning_rates = [5e-3, 1e-3, 1e-2]

loss_fn_args = {}
cap_aucs = []

start_epoch = 2

loss_caps = [1, 5, 10]
cap_calc = loss_fns.TripletLossWithAverage

for loss_cap in loss_caps:
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10): 
            print(loss_cap)
            model_aucs = []
            network = models.ConvNetWithEmbeddings(2)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(start_epoch):
                loss_fn_args['loss_cap'] = None
                _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
            for epoch in range(start_epoch, n_epochs + 1):
                loss_fn_args['loss_cap'] = loss_cap
                _, _ = train.train_triplet_capped_loss(epoch, train_loader_tripletloss_smote, network, optimizer, verbose=False, cap_calc=cap_calc, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)
        
    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
        
    cap_aucs.append([auc_mean, auc_variance])


for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = loss_caps[c]
    for i in range(len(learning_rates)): 
        row = ["triplet_loss_capped_smote_average", 2, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

1

Test set: Avg. loss: 0.0006964887678623199, AUC: 0.3482715


Test set: Avg. loss: 0.00073517307639122, AUC: 0.841398


Test set: Avg. loss: 0.0010314007997512817, AUC: 0.826527


Test set: Avg. loss: 0.0009717654287815094, AUC: 0.8537300000000001

1

Test set: Avg. loss: 0.0006961984634399415, AUC: 0.47539049999999994


Test set: Avg. loss: 0.0007382735610008239, AUC: 0.825611


Test set: Avg. loss: 0.0009567088186740875, AUC: 0.831588


Test set: Avg. loss: 0.0015623956918716432, AUC: 0.8068925

1

Test set: Avg. loss: 0.0006931769251823425, AUC: 0.509471


Test set: Avg. loss: 0.0006451610028743743, AUC: 0.8464630000000001


Test set: Avg. loss: 0.0012270017266273498, AUC: 0.8370259999999999


Test set: Avg. loss: 0.0013078323006629944, AUC: 0.864949

1

Test set: Avg. loss: 0.0006944571435451508, AUC: 0.445826


Test set: Avg. loss: 0.0008669923841953278, AUC: 0.8259850000000001


Test set: Avg. loss: 0.0009103761315345764, AUC: 0.8520369999999999


Test set: Avg. loss: 0.0009062

5

Test set: Avg. loss: 0.0006908078789710999, AUC: 0.6734819999999999


Test set: Avg. loss: 0.0006440731883049011, AUC: 0.8425530000000001


Test set: Avg. loss: 0.001008029878139496, AUC: 0.8353280000000001


Test set: Avg. loss: 0.0011021165251731873, AUC: 0.8578000000000001

5

Test set: Avg. loss: 0.0006938815414905548, AUC: 0.4701365


Test set: Avg. loss: 0.0006398897767066955, AUC: 0.823039


Test set: Avg. loss: 0.0009389999508857727, AUC: 0.8418840000000001


Test set: Avg. loss: 0.0012309846878051757, AUC: 0.839351

5

Test set: Avg. loss: 0.0006928956806659699, AUC: 0.5152815


Test set: Avg. loss: 0.0006695924997329711, AUC: 0.836996


Test set: Avg. loss: 0.000713811069726944, AUC: 0.869234


Test set: Avg. loss: 0.0009120012223720551, AUC: 0.884378

5

Test set: Avg. loss: 0.0006952646076679229, AUC: 0.454125


Test set: Avg. loss: 0.0007137651145458221, AUC: 0.819804


Test set: Avg. loss: 0.0005585785508155823, AUC: 0.794235


Test set: Avg. loss: 0.001033907383680343


Test set: Avg. loss: 0.0008677043914794922, AUC: 0.873766

10

Test set: Avg. loss: 0.0006948215365409851, AUC: 0.44974300000000006


Test set: Avg. loss: 0.0006209433376789094, AUC: 0.859944


Test set: Avg. loss: 0.0011180386543273926, AUC: 0.8403849999999999


Test set: Avg. loss: 0.0008133378028869629, AUC: 0.876189

10

Test set: Avg. loss: 0.0006891251504421235, AUC: 0.614881


Test set: Avg. loss: 0.0007206940054893493, AUC: 0.822441


Test set: Avg. loss: 0.000845925122499466, AUC: 0.8559945000000001


Test set: Avg. loss: 0.0010710387229919433, AUC: 0.867974

10

Test set: Avg. loss: 0.0006960428357124329, AUC: 0.439736


Test set: Avg. loss: 0.0005857323706150055, AUC: 0.841833


Test set: Avg. loss: 0.0009211321473121643, AUC: 0.846067


Test set: Avg. loss: 0.0010535757541656494, AUC: 0.8533420000000002

10

Test set: Avg. loss: 0.0006958933174610138, AUC: 0.37558400000000003


Test set: Avg. loss: 0.0007239218056201935, AUC: 0.830268


Test set: Avg. loss: 0.0007279396951

In [53]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [54]:
# capped smote using triplet loss w/ positive dist squared 

momentum=0
learning_rates = [5e-3, 1e-3, 1e-2]

loss_fn_args = {}


start_epoch = 2

loss_caps = [1, 5, 10]
cap_calc = loss_fns.TripletLoss

cap_aucs = []


for loss_cap in loss_caps:
    learning_rate_aucs = []
    for learning_rate in learning_rates:
        aucs = []
        for i in range(10): 
            print(loss_cap)
            model_aucs = []
            network = models.ConvNetWithEmbeddings(2)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(start_epoch):
                loss_fn_args['loss_cap'] = None
                _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
            for epoch in range(start_epoch, n_epochs + 1):
                loss_fn_args['loss_cap'] = loss_cap
                _, _ = train.train_triplet_capped_loss(epoch, train_loader_tripletloss_smote, network, optimizer, verbose=False, cap_calc=cap_calc, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)


    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    cap_aucs.append([auc_mean, auc_variance])



for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = loss_caps[c]
    for i in range(len(learning_rates)): 
        row = ["triplet_loss_capped_smote_pos_squared", 2, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

1

Test set: Avg. loss: 0.0006953137814998627, AUC: 0.41351000000000004


Test set: Avg. loss: 0.0012100113034248352, AUC: 0.751849


Test set: Avg. loss: 0.001347662389278412, AUC: 0.779907


Test set: Avg. loss: 0.0013431642651557923, AUC: 0.8068489999999999

1

Test set: Avg. loss: 0.000698688566684723, AUC: 0.35026250000000003


Test set: Avg. loss: 0.0011922909617424011, AUC: 0.7699495000000001


Test set: Avg. loss: 0.001296019196510315, AUC: 0.8160130000000001


Test set: Avg. loss: 0.0014232771992683411, AUC: 0.8320620000000001

1

Test set: Avg. loss: 0.0006938562393188477, AUC: 0.47384050000000005


Test set: Avg. loss: 0.0011157945990562439, AUC: 0.7417819999999999


Test set: Avg. loss: 0.0013826609253883362, AUC: 0.776867


Test set: Avg. loss: 0.0012448267340660095, AUC: 0.806332

1

Test set: Avg. loss: 0.000687132328748703, AUC: 0.732907


Test set: Avg. loss: 0.0013641536831855774, AUC: 0.7088535000000001


Test set: Avg. loss: 0.0014020829200744628, AUC: 0.75314799999


Test set: Avg. loss: 0.0012890581488609314, AUC: 0.818852

5

Test set: Avg. loss: 0.0006957947611808777, AUC: 0.4403390000000001


Test set: Avg. loss: 0.0013087846636772155, AUC: 0.7569419999999999


Test set: Avg. loss: 0.0014033986926078796, AUC: 0.7918499999999999


Test set: Avg. loss: 0.0011560410261154176, AUC: 0.82701

5

Test set: Avg. loss: 0.0006878388524055481, AUC: 0.7391975


Test set: Avg. loss: 0.001277011215686798, AUC: 0.733312


Test set: Avg. loss: 0.0013133590817451476, AUC: 0.7746850000000001


Test set: Avg. loss: 0.0013910502791404725, AUC: 0.7918069999999999

5

Test set: Avg. loss: 0.0006964699029922485, AUC: 0.3996565000000001


Test set: Avg. loss: 0.001165420174598694, AUC: 0.743013


Test set: Avg. loss: 0.0013641301393508911, AUC: 0.789986


Test set: Avg. loss: 0.0014104530215263366, AUC: 0.8106280000000001

5

Test set: Avg. loss: 0.0006948442161083221, AUC: 0.46871799999999997


Test set: Avg. loss: 0.0012675276398658752, AUC: 0.740739


Test set: Av

10

Test set: Avg. loss: 0.0006926549971103668, AUC: 0.532581


Test set: Avg. loss: 0.0011874104738235473, AUC: 0.76892


Test set: Avg. loss: 0.0012728224992752076, AUC: 0.8023020000000001


Test set: Avg. loss: 0.0012980905175209046, AUC: 0.826338

10

Test set: Avg. loss: 0.0007008664906024933, AUC: 0.3223985


Test set: Avg. loss: 0.0012494014501571655, AUC: 0.74944


Test set: Avg. loss: 0.0012630877494812013, AUC: 0.7997829999999999


Test set: Avg. loss: 0.0013315749168395995, AUC: 0.8174549999999999

10

Test set: Avg. loss: 0.0006991316080093384, AUC: 0.33853449999999996


Test set: Avg. loss: 0.0011583274602890014, AUC: 0.7944675000000001


Test set: Avg. loss: 0.0012185652256011964, AUC: 0.824557


Test set: Avg. loss: 0.001334098756313324, AUC: 0.8564489999999999

10

Test set: Avg. loss: 0.0006847937405109406, AUC: 0.705627


Test set: Avg. loss: 0.0011758599281311035, AUC: 0.7457389999999999


Test set: Avg. loss: 0.0012690225839614869, AUC: 0.793452


Test set: Avg. los

In [55]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []